In [1]:
import pandas as pd
df_matches = pd.read_csv("matches.csv")
df_matches.head(3)

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN


In [2]:
df_deliveries = pd.read_csv("deliveries.csv")
df_deliveries.head(3)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN


In [3]:
print(df_matches.shape)
print(df_deliveries.shape)

(756, 18)
(179078, 21)


In [4]:
print(df_matches.columns)
print(df_deliveries.columns)

Index(['id', 'season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2',
       'umpire3'],
      dtype='object')
Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')


In matches data frame columns like umpire1,umpire2,umpire3 are not useful because these days they are using technology. So we cant decide which umpire is more useful for predicting anything related to match.

Date is also not useful because we already have season column which is also almost equal to date.

City and venue also not important features for this year IPL. Because IPL2020 is happening in UAE, only 2014 ipl was happened in UAE that to half season. I am keeping only city and removing venue.

In [5]:
df_matches = df_matches.drop(['venue','date','umpire1','umpire2','umpire3'], axis = 1) 
df_matches.head(2)

,id,season,city,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match
0,1,2017,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh
1,2,2017,Pune,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith


In [6]:
print(df_matches["winner"].value_counts())

Mumbai Indians                 109
Chennai Super Kings            100
Kolkata Knight Riders           92
Royal Challengers Bangalore     84
Kings XI Punjab                 82
Rajasthan Royals                75
Delhi Daredevils                67
Sunrisers Hyderabad             58
Deccan Chargers                 29
Gujarat Lions                   13
Pune Warriors                   12
Rising Pune Supergiant          10
Delhi Capitals                  10
Kochi Tuskers Kerala             6
Rising Pune Supergiants          5
Name: winner, dtype: int64


In [7]:
#In 2019 delhi daredevils name changed to delhi capitals. So i am replacing old name with new name for all the rows.
df_matches = df_matches.replace(to_replace ="Delhi Daredevils", value = "Delhi Capitals") 

df_deliveries = df_deliveries.replace(to_replace = "Delhi Daredevils", value = "Delhi Capitals")

#In our dataset Rising pune supergiants name is in two ways. So iam replacing Rising Pune Supergiant with Rising Pune Supergiants
df_matches = df_matches.replace(to_replace ="Rising Pune Supergiant ", value = "Rising Pune Supergiants") 

df_deliveries = df_deliveries.replace(to_replace = "Rising Pune Supergiant", value = "Rising Pune Supergiants ")

In [8]:
#checking null values in matches dataset
print(df_matches.isnull().sum())

id                 0
season             0
city               7
team1              0
team2              0
toss_winner        0
toss_decision      0
result             0
dl_applied         0
winner             4
win_by_runs        0
win_by_wickets     0
player_of_match    4
dtype: int64


In [9]:
#printing null rows in matches dataset
df1_null = df_matches[df_matches.isna().any(axis=1)]
df1_null.head()

,id,season,city,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match
300,301,2011,Delhi,Delhi Capitals,Pune Warriors,Delhi Capitals,bat,no result,0,NaN,0,0,NaN
461,462,2014,NaN,Mumbai Indians,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Royal Challengers Bangalore,0,7,PA Patel
462,463,2014,NaN,Kolkata Knight Riders,Delhi Capitals,Kolkata Knight Riders,bat,normal,0,Delhi Capitals,0,4,JP Duminy
466,467,2014,NaN,Chennai Super Kings,Rajasthan Royals,Rajasthan Royals,field,normal,0,Chennai Super Kings,7,0,RA Jadeja
468,469,2014,NaN,Sunrisers Hyderabad,Delhi Capitals,Sunrisers Hyderabad,bat,normal,0,Sunrisers Hyderabad,4,0,AJ Finch


In [10]:
#After seeing above table we can clearly understood that 4 matches are not played beacuse of rain, badlight or pitch condition. 
#So i am replacing those null values in winner column with no result and palyer of the match column null values with not declared.
df_matches["winner"].fillna("No result", inplace = True)
df_matches['player_of_match'].fillna("Not declared", inplace = True)
#city column also has 7 null values. I checked those matches data in espn website and they played in Dubai.
df_matches["city"].fillna("Dubai", inplace = True)
print(df_matches.shape)

(756, 13)


In [11]:
#checking null values in deliveries dataset
print(df_deliveries.isnull().sum())

match_id                 0
inning                   0
batting_team             0
bowling_team             0
over                     0
ball                     0
batsman                  0
non_striker              0
bowler                   0
is_super_over            0
wide_runs                0
bye_runs                 0
legbye_runs              0
noball_runs              0
penalty_runs             0
batsman_runs             0
extra_runs               0
total_runs               0
player_dismissed    170244
dismissal_kind      170244
fielder             172630
dtype: int64


Player_dismissed, dismissal_kind and fielder are having null values. But if you consider today questions only player_dismissed is useful for calculating average of batsman. So i am removing remaining two features.

Filling player_dismissed null values with no wicket

In [12]:
#dropping columns
df_deliveries = df_deliveries.drop(['dismissal_kind','fielder'], axis = 1) 
#filling null values
df_deliveries["player_dismissed"].fillna("No Wicket", inplace = True)
df_deliveries.head(2)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,No Wicket
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,No Wicket


### Exploratory Data analysis(EDA)


### Q1) Who will in today match(CSK or DC)

In [13]:
#plotting DC and CSK wins against all teams
import plotly.graph_objs as go
df_winner = df_matches[['id','winner']].groupby('winner').size().reset_index()
df_winner.columns = ['winner','total_wins']
df_winner = df_winner.sort_values('total_wins',ascending=False)
df_winner = df_winner[(df_winner['winner'] == 'Delhi Capitals') | (df_winner['winner'] == 'Chennai Super Kings') ]

data = go.Bar(x = df_winner['winner'],y = df_winner['total_wins'])
layout = go.Layout(title = 'Total wins by DC and CSK in IPL',xaxis = dict(title = 'Team'),yaxis = dict(title = 'Number of wins'))
fig = go.Figure(data = data,layout = layout)
fig.show()

In [14]:
# Lets check winning ratio between MI VS KKR
df_DC_CSK = df_matches[((df_matches['team1'] == 'Delhi Capitals') | (df_matches['team1'] == 'Chennai Super Kings')) & 
                       ((df_matches['team2'] == 'Delhi Capitals') | (df_matches['team2'] == 'Chennai Super Kings')) ]
                       
                        

#plotting MI and KKR wins against each other
import plotly.graph_objs as go
df_winner = df_DC_CSK[['id','winner']].groupby('winner').size().reset_index()
df_winner.columns = ['winner','total_wins']
df_winner = df_winner.sort_values('total_wins',ascending=False)[:2]

data = go.Bar(x = df_winner['winner'],y = df_winner['total_wins'])
layout = go.Layout(title = 'Number of wins by DC and CSK against each other',xaxis = dict(title = 'Team'),yaxis = dict(title = 'Number of wins'))
fig = go.Figure(data = data,layout = layout)
fig.show()

When it comes to head-head between CSK and DC, CSK won more matches. But we know team will change for every two years, So lets look at individual players records.

In [15]:
#Lets look at how many player of the match awards won by current CSK and DC players.
#Note: I am taking top 8 palyers from both teams and taking data from last 5 seasons only because DC team players are mostly youngsters.
#Sometimes team win dependent on Number of match winners in your team. i.e why i am using this approach.
#Note: I am taking these player names because these players are played atleast one match for their respective sides in this season till now.
df_man_of_match_RCB = df_matches[((df_matches['player_of_match'] == 'SR Watson') | (df_matches['player_of_match'] == 'F du Plessis') |
                                (df_matches['player_of_match'] == 'MS Dhoni') | (df_matches['player_of_match'] == 'DL Chahar') |
                                (df_matches['player_of_match'] == 'SN Thakur') | (df_matches['player_of_match'] == 'RA Jadeja')) & 
                                (df_matches['season'] >= 2015)]
                                

print("Total number of man of matches won by present CSK players in last 5 seasons is", df_man_of_match_RCB.shape[0])

df_man_of_match_KXIP = df_matches[((df_matches['player_of_match'] == 'S Dhawan') | (df_matches['player_of_match'] == 'AM Rahane') |
                                (df_matches['player_of_match'] == 'SS Iyer') | (df_matches['player_of_match'] == 'RR Pant') |
                                (df_matches['player_of_match'] == 'K Rabada') | (df_matches['player_of_match'] == 'R Ashwin')) &
                                (df_matches['season'] >= 2015)]
                                
                                
print("Total number of man of matches won by present Delhi players in last 5 seasons is", df_man_of_match_KXIP.shape[0])

Total number of man of matches won by present CSK players in last 5 seasons is 18
Total number of man of matches won by present Delhi players in last 5 seasons is 22


Looking at above analysis Delhi players won more man of the match awards than CSK players in last 5 seasons. 

**Conclusion :-**

Delhi has more match winners than CSK at the same time as per my cricket knowledge delhi is young team, So i am predicting Delhi Capitals will win today match.


### Q2) How many runs will CSK score in the match

In [17]:
df_CSK = df_deliveries[df_deliveries['batting_team'] == "Chennai Super Kings"]
CSK_runs = df_CSK['total_runs'].sum()
no_of_matches = df_CSK['match_id'].nunique()
print(CSK_runs,"runs scored by CSK in",no_of_matches,"matches")

26418 runs scored by CSK in 164 matches


In [18]:
avg_runs = CSK_runs/no_of_matches
print("CSK Scored", int(avg_runs), "runs per match")

CSK Scored 161 runs per match


**Conclusion :-**

AS per above analysis i am predicting CSK will Score runs in the range of 151-165.

### Q3) How many wickets will CSK take from 91st ball to 120th ball.

First lets look at number of wickets taken by current CSK bowlers from 91st ball to 120th ball. Actually 91st to 120th balls means death overs. SO because of that i am taking the bowlers who will bowl at death overs only.

In [19]:
#filter out CSK bowlers and 91st to120th ball(i.e 16th over to 20th over)
df_wickets_by_CSKbowlers = df_deliveries[((df_deliveries['bowler'] == 'DJ Bravo') | (df_deliveries['bowler'] == 'SN Thakur') |
                                  (df_deliveries['bowler'] == 'DL Chahar') | (df_deliveries['bowler'] == 'SM Curran')) 
                                                                 &
                                  (df_deliveries['over'] >= 16)]

no_of_wickets =len(df_wickets_by_CSKbowlers[df_wickets_by_CSKbowlers['player_dismissed'] != 'No Wicket'])
print(no_of_wickets, " wickets taken by present CSK bowlers in",df_wickets_by_CSKbowlers['match_id'].nunique(), "matches in death overs(16-20).")                          

127  wickets taken by present CSK bowlers in 117 matches in death overs(16-20).


In [20]:
#Lets look at how many wickets took CSK team from 91st ball to 120th ball.
df_wickets_by_CSK = df_deliveries[(df_deliveries['bowling_team'] == 'Chennai Super Kings') & (df_deliveries['over'] >= 16)] 

no_of_wickets =len(df_wickets_by_CSK[df_wickets_by_CSK['player_dismissed'] != 'No Wicket'])
print(no_of_wickets, " wickets taken by CSK team in",df_wickets_by_CSK['match_id'].nunique(), "matches in death overs(16-20).")

384  wickets taken by CSK team in 157 matches in death overs(16-20).


In [21]:
#Lets look at how many wickets delhi capitals lost in death overs(91st ball to 120th ball)

df_wickets_Delhi = df_deliveries[(df_deliveries['bowling_team'] == 'Delhi Capitals') & (df_deliveries['over'] >= 16)] 

no_of_wickets =len(df_wickets_Delhi[df_wickets_Delhi['player_dismissed'] != 'No Wicket'])
print(no_of_wickets, " wickets lost by Delhi capitals team in",df_wickets_Delhi['match_id'].nunique(), "matches in death overs(16-20).")

393  wickets lost by Delhi capitals team in 163 matches in death overs(16-20).


**Conclusion :-**

By looking at above analysis we can say CSK will take 2-3 wickets from 91st ball to 120th ball. So i am predicting they will take atleast 3 wickets in death overs.

### Q4) How many balls will CSK take to Score their 100 runs

In [65]:
# import numpy as np
# df_CSK_batting = df_deliveries[(df_deliveries['batting_team'] == 'Chennai Super Kings')]
# df_CSK_batting.head()

# runs_list = df_CSK_batting['total_runs'].to_list()
# run_cum = np.cumsum(runs_list)
# match_list =  df_CSK_batting['match_id'].to_list()
# balls = df_CSK_batting['ball'].to_list()
# no_of_matches = len(np.unique(match_list))
# print(no_of_matches)
# dictionary_runs_matchid = dict(zip(match_list, run_cum)) 
# print(dictionary_runs_matchid)

164
{61: 240, 67: 448, 71: 600, 74: 778, 79: 947, 83: 1056, 85: 1200, 87: 1388, 91: 1569, 96: 1725, 103: 1780, 107: 1892, 110: 2093, 114: 2241, 116: 2357, 117: 2520, 118: 2666, 122: 2845, 125: 3025, 131: 3190, 137: 3354, 141: 3517, 144: 3695, 149: 3880, 152: 4021, 159: 4150, 162: 4301, 166: 4489, 169: 4605, 173: 4751, 179: 4910, 182: 5074, 185: 5264, 190: 5409, 192: 5544, 194: 5724, 198: 5884, 202: 6046, 206: 6292, 211: 6457, 216: 6595, 222: 6738, 224: 6850, 228: 7045, 232: 7187, 234: 7355, 235: 7508, 243: 7696, 248: 7879, 252: 8010, 257: 8166, 262: 8308, 264: 8453, 272: 8618, 276: 8767, 281: 8881, 285: 9077, 289: 9253, 297: 9405, 302: 9533, 304: 9710, 307: 9915, 308: 10027, 313: 10220, 318: 10330, 320: 10538, 324: 10693, 332: 10857, 334: 11004, 343: 11153, 347: 11292, 352: 11452, 355: 11625, 361: 11752, 364: 11867, 368: 12027, 371: 12147, 379: 12334, 380: 12556, 381: 12746, 386: 12885, 391: 13024, 396: 13190, 399: 13325, 404: 13494, 406: 13618, 410: 13804, 414: 13964, 418: 14164, 422:

In [84]:
# df_CSK_batting = df_deliveries[(df_deliveries['batting_team'] == 'Chennai Super Kings')]
# df_CSK_batting.head()
# df = df_CSK_batting[['match_id', 'total_runs']] 

# #I am calculating what is the average runs scored by CSK per over and stored in total_runs column
# df.groupby(['match_id']).sum()
# df.head(2)

,match_id,total_runs
14087,61,0
14088,61,0


I tried in multiple ways but i am unable to write code for this question.  

### Q5) How many No-balls will be bowled in the match

In [42]:
#For no ball question i am filtering out the bowlers in both the team and checking how many no balls they bowled.
#Here i am not picking Anrich Nortje and young bowlers. I dont think they played ipl before this season except 1 or 2 matches.
df_DC_CSK_bowlers = df_deliveries[((df_deliveries['bowler'] == 'K Rabada') | (df_deliveries['bowler'] == 'R Ashwin') |
                                   (df_deliveries['bowler'] == 'AR Patel') |(df_deliveries['bowler'] == 'DL Chahar	') | 
                                   (df_deliveries['bowler'] == 'SN Thakur') |(df_deliveries['bowler'] == 'RA Jadeja'))]
                                      
                               
                            
print("Number of no balls bowled  by present DC and CSK bowlers : ",df_DC_CSK_bowlers['noball_runs'].sum())
print("Number of balls bowled by present DC and CSK bowlers : ", df_DC_CSK_bowlers.shape[0])

Number of no balls bowled  by present DC and CSK bowlers :  6
Number of balls bowled by present DC and CSK bowlers :  8481


In [44]:
#Lets convert number of balls into overs
no_of_overs = df_DC_CSK_bowlers.shape[0]//6

#We know every match has total 40 overs(20 overs per side).
#Here i am calculating number of matches will took to complte above number of overs.
no_of_matches = no_of_overs/40

print(int(no_of_matches), "matches will take to complete", no_of_overs, "overs in T20")

35 matches will take to complete 1413 overs in T20


**Conclusion :-**

By looking at above analysis the current CSK and DC bowlers bowled only 6 No-balls in 35 matches(1413 overs).

I am predicting in today match they will bowl 0-1 No-balls.